# Kosmetyki męskie - część I - Preprocessing

### Wstęp

Prezentowane na tej stronie materiały są zapisem drugiego etapu projektu "Kosmetyki męskie". 

Opis projektu: 

I etap (pobranie danych):

Dane pobrane w pierwszym etapie projektu zostały zapisane jako *Data Frame* w pliku .pkl (*pickle*).

Celem tego etapu jest przygotowanie danych (sprawdzenie integralności, czyszczenie, konwersja).

#### Pobranie niezbędnych bibliotek

In [44]:
import pandas as pd

#### Podstawowe informacje o danych

In [45]:
df = pd.read_pickle("../data/male_products_data.pkl")
print(f'Dane posiadają {df.shape[0]} wierszy i {df.shape[1]} kolumn.')

Dane posiadają 1794 wierszy i 19 kolumn.


In [46]:
df.head(3)

,product_id,name,caption,cat_1,cat_2,cat_3,cat_4,cat_5,url,capacity,current_price_pln,regular_price_pln,lowest_price_last_30_days_pln,price_per_unit_pln,unit,mega_badge,ingredients,rating,reviews
0,2106072,RITUALS... Homme,Balsam do stylizacji brody,Pielęgnacja i higiena,Broda i zarost,Mycie i pielęgnacja zarostu,Balsamy i olejki do brody,,https://www.rossmann.pl/Produkt/Balsamy-i-olej...,30 ml,74.99,,93.99,249.97,100 ml,1,"Ingredients: Ingredients: Aqua/Water, Hydrogen...",,
1,2095846,WILKINSON SWORD Hydro 5,Maszynka do golenia rączka + 9 wkładów,Pielęgnacja i higiena,Broda i zarost,Golenie zarostu,Maszynki tradycyjne,,https://www.rossmann.pl/Produkt/Maszynki-trady...,1 szt.,79.99,,119.99,79.99,1 szt.,1,"PEG-115M, Polyquaternium-10, Silica [Nano], Aq...",,
2,2083791,JAGUAR Era Reloaded,"Zestaw dla mężczyzn, woda toaletowa 100ml + że...",Mężczyzna,Perfumy męskie,Zestawy zapachowe,,,https://www.rossmann.pl/Produkt/Zestawy-zapach...,1 szt.,154.99,,,154.99,1 szt.,0,"Alcohol Denat., Parfum (Fragrance), Aqua (Wate...",4.5,2


#### Sprawdzenie czy występują w danych jakieś puste kolumny i wiersze.

In [47]:
# Czy występują puste koloumny?
df.isna().all()

product_id                       False
name                             False
caption                          False
cat_1                            False
cat_2                            False
cat_3                            False
cat_4                            False
cat_5                            False
url                              False
capacity                         False
current_price_pln                False
regular_price_pln                False
lowest_price_last_30_days_pln    False
price_per_unit_pln               False
unit                             False
mega_badge                       False
ingredients                      False
rating                           False
reviews                          False
dtype: bool

In [48]:
# Czy występują puste wiersze?
empty_rows = df[df.isna().all(axis=1)]
# Wypisuje listę pustych indeksów
print(empty_rows.index.tolist())

[]


#### Sprawdzenie i konwersja typów danych.

In [49]:
# Typy danych
df.dtypes

product_id                        object
name                              object
caption                           object
cat_1                             object
cat_2                             object
cat_3                             object
cat_4                             object
cat_5                             object
url                               object
capacity                          object
current_price_pln                float64
regular_price_pln                 object
lowest_price_last_30_days_pln     object
price_per_unit_pln               float64
unit                              object
mega_badge                         int64
ingredients                       object
rating                            object
reviews                           object
dtype: object

In [50]:
# Konwersja typów danych
df['product_id'] = df['product_id'].astype(int)
df['name'] = df['name'].astype(pd.StringDtype())
df['caption'] = df['caption'].astype(pd.StringDtype())
df['cat_1'] = df['cat_1'].astype(pd.StringDtype())
df['cat_2'] = df['cat_2'].astype(pd.StringDtype())
df['cat_3'] = df['cat_3'].astype(pd.StringDtype())
df['cat_4'] = df['cat_4'].astype(pd.StringDtype())
df['cat_5'] = df['cat_5'].astype(pd.StringDtype())
df['url'] = df['url'].astype(pd.StringDtype())
df['capacity'] = df['capacity'].astype(pd.StringDtype())
df['unit'] = df['unit'].astype(pd.StringDtype())
df['ingredients'] = df['ingredients'].astype(pd.StringDtype())
df["regular_price_pln"] = pd.to_numeric(df["regular_price_pln"], errors="coerce")
df["lowest_price_last_30_days_pln"] = pd.to_numeric(df["lowest_price_last_30_days_pln"], errors="coerce")
df['mega_badge'] = df['mega_badge'].astype(bool)
df["rating"] = pd.to_numeric(df["rating"], errors="coerce")
df["reviews"] = pd.to_numeric(df["reviews"], errors="coerce")

df.dtypes

product_id                                int64
name                             string[python]
caption                          string[python]
cat_1                            string[python]
cat_2                            string[python]
cat_3                            string[python]
cat_4                            string[python]
cat_5                            string[python]
url                              string[python]
capacity                         string[python]
current_price_pln                       float64
regular_price_pln                       float64
lowest_price_last_30_days_pln           float64
price_per_unit_pln                      float64
unit                             string[python]
mega_badge                                 bool
ingredients                      string[python]
rating                                  float64
reviews                                 float64
dtype: object

#### Sprawdzenie unikalności kolumny product_id

In [51]:
df['product_id'].is_unique

True

#### Rozdzielenie wartości i jednostek w kolumnach *capacity* i *unit*
Podział na capacity_value, capacity_unit, price_reference_value, price_reference_unit a price_per_unit_pln zamieniam na price_reference_pln

In [52]:
df['capacity_value'] = df['capacity'].str.extract(r'(\d+)').astype(int)
df['capacity_unit'] = df['capacity'].str.extract(r'([a-zA-ZżźćńółęąśŻŹĆĄŚĘŁÓŃ]+)')
df['price_reference_value'] = df['unit'].str.extract(r'(\d+)').astype(int)
df['price_reference_unit'] = df['unit'].str.extract(r'([a-zA-ZżźćńółęąśŻŹĆĄŚĘŁÓŃ]+)')
df['price_reference_pln'] = df['price_per_unit_pln']

df = df.drop(columns=['capacity', 'unit', 'price_per_unit_pln'])

df.head(3)

,product_id,name,caption,cat_1,cat_2,cat_3,cat_4,cat_5,url,current_price_pln,...,lowest_price_last_30_days_pln,mega_badge,ingredients,rating,reviews,capacity_value,capacity_unit,price_reference_value,price_reference_unit,price_reference_pln
0,2106072,RITUALS... Homme,Balsam do stylizacji brody,Pielęgnacja i higiena,Broda i zarost,Mycie i pielęgnacja zarostu,Balsamy i olejki do brody,,https://www.rossmann.pl/Produkt/Balsamy-i-olej...,74.99,...,93.99,True,"Ingredients: Ingredients: Aqua/Water, Hydrogen...",NaN,NaN,30,ml,100,ml,249.97
1,2095846,WILKINSON SWORD Hydro 5,Maszynka do golenia rączka + 9 wkładów,Pielęgnacja i higiena,Broda i zarost,Golenie zarostu,Maszynki tradycyjne,,https://www.rossmann.pl/Produkt/Maszynki-trady...,79.99,...,119.99,True,"PEG-115M, Polyquaternium-10, Silica [Nano], Aq...",NaN,NaN,1,szt,1,szt,79.99
2,2083791,JAGUAR Era Reloaded,"Zestaw dla mężczyzn, woda toaletowa 100ml + że...",Mężczyzna,Perfumy męskie,Zestawy zapachowe,,,https://www.rossmann.pl/Produkt/Zestawy-zapach...,154.99,...,NaN,False,"Alcohol Denat., Parfum (Fragrance), Aqua (Wate...",4.5,2.0,1,szt,1,szt,154.99


#### Weryfikacja wartości w kolumnach *capacity_unit* i *price_reference_unit*

In [53]:
df['capacity_unit'].value_counts()

capacity_unit
ml        1374
szt        315
para        62
g           27
pary         9
x            6
gratis       1
Name: count, dtype: Int64

In [54]:
df[df['capacity_unit'].isin(['x', 'gratis'])]

,product_id,name,caption,cat_1,cat_2,cat_3,cat_4,cat_5,url,current_price_pln,...,lowest_price_last_30_days_pln,mega_badge,ingredients,rating,reviews,capacity_value,capacity_unit,price_reference_value,price_reference_unit,price_reference_pln
202,2111090,GILLETTE Mach 3,"Wkłady do maszynki do golenia, 3-ostrzowe, dla...",Pielęgnacja i higiena,Broda i zarost,Golenie zarostu,Wkłady i ostrza do maszynek,,https://www.rossmann.pl/Produkt/Wklady-i-ostrz...,119.99,...,169.99,True,"PEG-45M, PEG-180M, PEG-100, PEG-7M, Methyl Di-...",NaN,NaN,3,x,1,szt,8.00
253,2110984,GILLETTE Fusion 5,"Wkłady do maszynki do golenia 5-ostrzowe, dla ...",Pielęgnacja i higiena,Broda i zarost,Golenie zarostu,Wkłady i ostrza do maszynek,,https://www.rossmann.pl/Produkt/Wklady-i-ostrz...,139.99,...,199.99,True,"PEG-115M, PEG-7M, PEG-100, Silica, Tocopheryl ...",NaN,NaN,3,x,1,szt,11.67
258,245532,SEBORADIN Forte,"Ampułki do włosów, przyspieszające odrost, unisex",Włosy,Skóra głowy,Serum i wcierki do skóry głowy,,,https://www.rossmann.pl/Produkt/Serum-i-wcierk...,41.99,...,48.99,False,"Ingredients: Aqua, Alcohol Denat., Propylene G...",5.0,3.0,14,x,100,ml,54.53
473,108416,WILKINSON SWORD Xtreme 3 Sensitive,"Maszynki do golenia, 3-ostrzowe, jednorazowe, ...",Pielęgnacja i higiena,Broda i zarost,Golenie zarostu,Maszynki tradycyjne,,https://www.rossmann.pl/Produkt/Maszynki-trady...,15.49,...,15.99,True,"PEG-115M, PVP, PEG-100, Cyclodextrin, Tocopher...",4.8,43.0,6,gratis,1,szt,2.58
570,2099093,L'BIOTICA BIOVAX Trychologic Advanced Detox,"Ampułki do włosów, kuracja redukująca przetłus...",Włosy,Skóra głowy,Serum i wcierki do skóry głowy,,,https://www.rossmann.pl/Produkt/Serum-i-wcierk...,79.99,...,NaN,False,"Ingredients: Aqua, Niacinamide, Propanediol, G...",NaN,NaN,7,x,100,ml,228.54
1734,2073406,DERMENA,"Kuracja do włosów w ampułkach, przeciw wypadaniu",Włosy,Skóra głowy,Serum i wcierki do skóry głowy,,,https://www.rossmann.pl/Produkt/Serum-i-wcierk...,49.99,...,NaN,False,"Aqua, Alcohol, Pentylene Glycol, Isopropyl Alc...",4.8,13.0,5,x,100,ml,142.83
1750,2099092,L'BIOTICA BIOVAX Trychologic,"Ampułki do włosów, kuracja dla mężczyzn",Mężczyzna,Włosy,Pielęgnacja skóry głowy,,,https://www.rossmann.pl/Produkt/Pielegnacja-sk...,79.99,...,NaN,False,"Ingredients: Aqua, Alcohol Denat., Propylene G...",5.0,4.0,7,x,100,ml,228.54


In [55]:
unit_mapping = {
    'para': 'szt',
    'pary': 'szt',
    'x': 'szt',
    'gratis': 'szt'
}
df['capacity_unit'] = df['capacity_unit'].replace(unit_mapping)

df['capacity_unit'].value_counts()

capacity_unit
ml     1374
szt     393
g        27
Name: count, dtype: Int64

In [56]:
df['price_reference_unit'].value_counts()

price_reference_unit
ml     1378
szt     389
g        27
Name: count, dtype: Int64

#### Weryfikacja danych w kolumnach liczbowych

In [57]:
df.describe()

,product_id,current_price_pln,regular_price_pln,lowest_price_last_30_days_pln,rating,reviews,capacity_value,price_reference_value,price_reference_pln
count,1.794000e+03,1794.000000,171.000000,533.000000,1059.000000,1059.000000,1794.000000,1794.000000,1794.00000
mean,1.116550e+06,130.268094,52.965439,97.232402,4.820019,10.140699,93.658305,78.533445,157.91903
std,8.985746e+05,175.204709,61.454278,117.413026,0.347983,11.589874,102.718019,40.808139,217.66709
min,3.180000e+02,2.190000,2.790000,2.190000,1.000000,1.000000,1.000000,1.000000,0.67000
25%,3.193058e+05,19.990000,17.490000,19.990000,4.800000,2.000000,30.000000,100.000000,16.99000
50%,4.200335e+05,51.490000,24.990000,51.990000,4.900000,6.000000,75.000000,100.000000,63.98000
75%,2.088604e+06,169.990000,53.990000,121.990000,5.000000,14.000000,100.000000,100.000000,229.99000
max,2.114179e+06,1099.990000,349.990000,1119.990000,5.000000,105.000000,900.000000,100.000000,1799.98000


**current_price_pln** - wszystkie produkty mają cenę, zakres wygląda prawidłowo ✅

**regular_price_pln** - 171 produktów posiada cenę katalogową, zakres wygląda prawidłowo ✅

**lowest_price_last_30_days_pln** - 533 produkty posiadają najniższą cenę za 30 dni, zakres wygląda prawidłowo ✅

**rating** - 1059 produktów posiada rating, dane mocno skrzywione (średnia 4.8) ✅

**reviews** - tyle samo produktów posiada oceny użytkowników, zakres wygląda ok (1-105) ✅

**capacity_value** - wszystkie produkty posiadają wielkość, zakresy ok ✅

**price_reference_value** - wszystkie produkty posiadają wielkość referencyjną, zakres ok ✅

**price_reference_pln** - wszystkie produkty posiadają cenę referencyjną, zakres ok ✅

#### Przegląd kategorii produktów

Do analizy potrzebuję produktów z kategorii kosmetyków. W kategorii mężczyzna znajdują się także inne produkty, które muszę odsiać.

In [58]:
# Przegląd kategorii pierwszego poziomu
df['cat_1'].value_counts()

cat_1
Mężczyzna                890
Pielęgnacja i higiena    367
Perfumy                  351
Lifestyle                104
Włosy                     82
Name: count, dtype: Int64

In [59]:
# Przegląd kategorii drugiego poziomu w kategorii Lifestyle
df[df['cat_1'] == 'Lifestyle']['cat_2'].value_counts()

cat_2
Odzież i dodatki    98
Podróż               6
Name: count, dtype: Int64

In [60]:
# Weryfikacja kategorii Podróż
df[df['cat_2'] == 'Podróż']['cat_3']

427     Minikosmetyki
495     Minikosmetyki
749     Minikosmetyki
1062    Minikosmetyki
1602    Minikosmetyki
1729    Minikosmetyki
Name: cat_3, dtype: string

In [61]:
# Usunięcie produktów z kategorii Lifestyle > Odzież i dodatki
df = df.query('not (cat_1 == "Lifestyle" and cat_2 == "Odzież i dodatki")')
df[df['cat_1'] == 'Lifestyle']['cat_2'].value_counts()

cat_2
Podróż    6
Name: count, dtype: Int64

In [62]:
# Przegląd kategorii drugiego poziomu w kategorii Mężczyzna
df[df['cat_1'] == 'Mężczyzna']['cat_2'].value_counts()

cat_2
Perfumy męskie    505
Higiena           107
Odzież męska      105
Pielęgnacja        90
Golenie            41
Włosy              31
Zarost             11
Name: count, dtype: Int64

In [63]:
# Usunięcie produktów w kategorii Mężczyzna > Odzież męska
df = df.query('not (cat_1 == "Mężczyzna" and cat_2 == "Odzież męska")')
df[df['cat_1'] == 'Mężczyzna']['cat_2'].value_counts()

cat_2
Perfumy męskie    505
Higiena           107
Pielęgnacja        90
Golenie            41
Włosy              31
Zarost             11
Name: count, dtype: Int64

In [64]:
# Przegląd podkategorii Mężczyzna > Zarost
df.query('cat_1 == "Mężczyzna" and cat_2 == "Zarost"')

,product_id,name,caption,cat_1,cat_2,cat_3,cat_4,cat_5,url,current_price_pln,...,lowest_price_last_30_days_pln,mega_badge,ingredients,rating,reviews,capacity_value,capacity_unit,price_reference_value,price_reference_unit,price_reference_pln
236,2085517,EWA SCHMITT Beard Master,Szczotka do brody,Mężczyzna,Zarost,Akcesoria do zarostu,,,https://www.rossmann.pl/Produkt/Akcesoria-do-z...,16.99,...,NaN,False,"Drewno bukowe, włosia dzika.",5.0,5.0,1,szt,1,szt,16.99
325,270396,ISANA MEN,"Pomada do brody, z Olejem Arganowym i Woskiem ...",Mężczyzna,Zarost,Pielęgnacja zarostu,,,https://www.rossmann.pl/Produkt/Pielegnacja-za...,27.99,...,NaN,False,"Aqua, Cera Alba, Isopropyl Palmitate, Copernic...",5.0,2.0,50,ml,100,ml,55.98
418,2078192,AA MEN Beard,Koncentrat na porost brody i wąsów,Mężczyzna,Zarost,Pielęgnacja zarostu,,,https://www.rossmann.pl/Produkt/Pielegnacja-za...,33.99,...,NaN,False,"Aqua, Dimethicone, Glycerin, Pentylene Glycol,...",5.0,4.0,30,ml,100,ml,113.30
572,2085518,BARBERO,Booster wzmacniający brodę i włosy,Mężczyzna,Zarost,Pielęgnacja zarostu,,,https://www.rossmann.pl/Produkt/Pielegnacja-za...,31.99,...,NaN,False,"Aqua, Alcohol Denat., PEG-40 Hydrogenated Cast...",5.0,1.0,150,ml,100,ml,21.33
711,339519,BARBERO,"Pomada do stylizacji wąsów, brody i włosów",Mężczyzna,Zarost,Stylizacja zarostu,,,https://www.rossmann.pl/Produkt/Stylizacja-zar...,37.99,...,NaN,False,"Ingredients: Aqua, Lanolin Cera, Petrolanum, C...",4.0,1.0,40,g,100,g,94.98
744,267581,TOŁPA Dermo Barber.,"Krem do brody, skóra wrażliwa, alergiczna",Mężczyzna,Zarost,Stylizacja zarostu,,,https://www.rossmann.pl/Produkt/Stylizacja-zar...,32.99,...,NaN,False,"Ingredients: Aqua, Propylene Glycol, Glycerin,...",5.0,2.0,50,ml,100,ml,65.98
759,419910,HAGI Men Whisky Barber,Pomada do brody i wąsów,Mężczyzna,Zarost,Stylizacja zarostu,,,https://www.rossmann.pl/Produkt/Stylizacja-zar...,49.99,...,NaN,False,Ingredients: Brassica Campestris (Rapeseed) Se...,4.0,1.0,50,ml,100,ml,99.98
1185,270391,ISANA MEN,"Żel do brody, z ekstraktem z Guarany",Mężczyzna,Zarost,Pielęgnacja zarostu,,,https://www.rossmann.pl/Produkt/Pielegnacja-za...,19.99,...,NaN,False,"Ingredients: Aqua, Glycerin, Dicaprylyl Ether,...",5.0,4.0,50,ml,100,ml,39.98
1291,396104,ARGANOVE Cool Exotic,"Balsam do stylizacji zarostu, pielęgnacyjny, M...",Mężczyzna,Zarost,Stylizacja zarostu,,,https://www.rossmann.pl/Produkt/Stylizacja-zar...,29.99,...,NaN,False,Ingredients: INCI: Prunus Amygdalus Dulcis Oil...,4.5,6.0,50,ml,100,ml,59.98
1370,2100676,RITUALS... Homme,"Olejek do brody, Olejek arganowy i Bazyliowy",Mężczyzna,Zarost,Pielęgnacja zarostu,,,https://www.rossmann.pl/Produkt/Pielegnacja-za...,74.99,...,93.99,True,Ingredients: Prunus Armeniaca (Apricot) Kernel...,NaN,NaN,30,ml,100,ml,249.97


In [65]:
# Usunięcie produktu 2085517 (Szczotka do brody)
df = df.query('product_id != 2085517')
df.query('cat_1 == "Mężczyzna" and cat_2 == "Zarost"')


,product_id,name,caption,cat_1,cat_2,cat_3,cat_4,cat_5,url,current_price_pln,...,lowest_price_last_30_days_pln,mega_badge,ingredients,rating,reviews,capacity_value,capacity_unit,price_reference_value,price_reference_unit,price_reference_pln
325,270396,ISANA MEN,"Pomada do brody, z Olejem Arganowym i Woskiem ...",Mężczyzna,Zarost,Pielęgnacja zarostu,,,https://www.rossmann.pl/Produkt/Pielegnacja-za...,27.99,...,NaN,False,"Aqua, Cera Alba, Isopropyl Palmitate, Copernic...",5.0,2.0,50,ml,100,ml,55.98
418,2078192,AA MEN Beard,Koncentrat na porost brody i wąsów,Mężczyzna,Zarost,Pielęgnacja zarostu,,,https://www.rossmann.pl/Produkt/Pielegnacja-za...,33.99,...,NaN,False,"Aqua, Dimethicone, Glycerin, Pentylene Glycol,...",5.0,4.0,30,ml,100,ml,113.30
572,2085518,BARBERO,Booster wzmacniający brodę i włosy,Mężczyzna,Zarost,Pielęgnacja zarostu,,,https://www.rossmann.pl/Produkt/Pielegnacja-za...,31.99,...,NaN,False,"Aqua, Alcohol Denat., PEG-40 Hydrogenated Cast...",5.0,1.0,150,ml,100,ml,21.33
711,339519,BARBERO,"Pomada do stylizacji wąsów, brody i włosów",Mężczyzna,Zarost,Stylizacja zarostu,,,https://www.rossmann.pl/Produkt/Stylizacja-zar...,37.99,...,NaN,False,"Ingredients: Aqua, Lanolin Cera, Petrolanum, C...",4.0,1.0,40,g,100,g,94.98
744,267581,TOŁPA Dermo Barber.,"Krem do brody, skóra wrażliwa, alergiczna",Mężczyzna,Zarost,Stylizacja zarostu,,,https://www.rossmann.pl/Produkt/Stylizacja-zar...,32.99,...,NaN,False,"Ingredients: Aqua, Propylene Glycol, Glycerin,...",5.0,2.0,50,ml,100,ml,65.98
759,419910,HAGI Men Whisky Barber,Pomada do brody i wąsów,Mężczyzna,Zarost,Stylizacja zarostu,,,https://www.rossmann.pl/Produkt/Stylizacja-zar...,49.99,...,NaN,False,Ingredients: Brassica Campestris (Rapeseed) Se...,4.0,1.0,50,ml,100,ml,99.98
1185,270391,ISANA MEN,"Żel do brody, z ekstraktem z Guarany",Mężczyzna,Zarost,Pielęgnacja zarostu,,,https://www.rossmann.pl/Produkt/Pielegnacja-za...,19.99,...,NaN,False,"Ingredients: Aqua, Glycerin, Dicaprylyl Ether,...",5.0,4.0,50,ml,100,ml,39.98
1291,396104,ARGANOVE Cool Exotic,"Balsam do stylizacji zarostu, pielęgnacyjny, M...",Mężczyzna,Zarost,Stylizacja zarostu,,,https://www.rossmann.pl/Produkt/Stylizacja-zar...,29.99,...,NaN,False,Ingredients: INCI: Prunus Amygdalus Dulcis Oil...,4.5,6.0,50,ml,100,ml,59.98
1370,2100676,RITUALS... Homme,"Olejek do brody, Olejek arganowy i Bazyliowy",Mężczyzna,Zarost,Pielęgnacja zarostu,,,https://www.rossmann.pl/Produkt/Pielegnacja-za...,74.99,...,93.99,True,Ingredients: Prunus Armeniaca (Apricot) Kernel...,NaN,NaN,30,ml,100,ml,249.97
1582,2100671,RITUALS... Homme,"Szampon i odżywka do brody 2w1, Olejek arganow...",Mężczyzna,Zarost,Szampony do brody,,,https://www.rossmann.pl/Produkt/Szampony-do-br...,37.99,...,47.99,True,"Ingredients: Aqua/Water, Coco-Glucoside, Sodiu...",NaN,NaN,250,ml,100,ml,15.20


In [66]:
# Przegląd podkategorii Mężczyzna > Włosy
df.query('cat_1 == "Mężczyzna" and cat_2 == "Włosy"')['cat_3'].value_counts()

cat_3
Szampony i odżywki              16
Koloryzacja włosów i zarostu     9
Stylizacja włosów                4
Akcesoria do włosów              1
Pielęgnacja skóry głowy          1
Name: count, dtype: Int64

In [67]:
# Usunięcie kategorii Akcesoria do włosów
df = df.query('not(cat_1 == "Mężczyzna" and cat_2 == "Włosy" and cat_3 == "Akcesoria do włosów")')
df.query('cat_1 == "Mężczyzna" and cat_2 == "Włosy"')['cat_3'].value_counts()

cat_3
Szampony i odżywki              16
Koloryzacja włosów i zarostu     9
Stylizacja włosów                4
Pielęgnacja skóry głowy          1
Name: count, dtype: Int64

In [68]:
# Przegląd podkategorii Mężczyzna > Higiena
df.query('cat_1 == "Mężczyzna" and cat_2 == "Higiena"')['cat_3'].value_counts()

cat_3
Dezodoranty i antyperspiranty    60
Żele pod prysznic                45
Wkładki higienicze                1
Płyny do higieny intymnej         1
Name: count, dtype: Int64

In [69]:
# Usunięcie kategorii Wkładki higienicze
df = df.query('not(cat_1 == "Mężczyzna" and cat_2 == "Higiena" and cat_3 == "Wkładki higienicze")')
df.query('cat_1 == "Mężczyzna" and cat_2 == "Higiena"')['cat_3'].value_counts()

cat_3
Dezodoranty i antyperspiranty    60
Żele pod prysznic                45
Płyny do higieny intymnej         1
Name: count, dtype: Int64

In [70]:
# Przegląd podkategorii Mężczyzna > Pielęgnacja
df.query('cat_1 == "Mężczyzna" and cat_2 == "Pielęgnacja"')['cat_3'].value_counts()

cat_3
Pielęgnacja twarzy     70
Oczyszczanie twarzy    20
Name: count, dtype: Int64

In [71]:
# Przegląd produktów w kategorii Mężczyzna > Pielęgnacja, które są na sztuki
df.query('cat_1 == "Mężczyzna" and cat_2 == "Pielęgnacja" and capacity_unit == "szt"')

,product_id,name,caption,cat_1,cat_2,cat_3,cat_4,cat_5,url,current_price_pln,...,lowest_price_last_30_days_pln,mega_badge,ingredients,rating,reviews,capacity_value,capacity_unit,price_reference_value,price_reference_unit,price_reference_pln
182,2096264,RITUALS... Homme,Zestaw zapach do samochodu + sportowy żel pod ...,Mężczyzna,Pielęgnacja,Oczyszczanie twarzy,,,https://www.rossmann.pl/Produkt/Oczyszczanie-t...,187.99,...,234.99,True,,NaN,NaN,1,szt,1,szt,187.99
290,2094631,ISANA MEN,"Płatki pod oczy, hydrożelowe, nawilżająco-kojące",Mężczyzna,Pielęgnacja,Pielęgnacja twarzy,,,https://www.rossmann.pl/Produkt/Pielegnacja-tw...,4.59,...,NaN,False,"Ingredients: Aqua, Glycerin, Sorbitol, Trehalo...",4.7,20.0,2,szt,1,szt,2.30
866,2098656,NUXE Men Exclusively,"Zestaw kosmetyków dla mężczyzn, żel nawilżając...",Mężczyzna,Pielęgnacja,Pielęgnacja twarzy,,,https://www.rossmann.pl/Produkt/Pielegnacja-tw...,149.99,...,NaN,True,Ingredients: Żel pod prysznic: Lauryl Glucose ...,NaN,NaN,1,szt,1,szt,149.99
1142,2094630,ISANA MEN,"Płatki pod oczy, hydrożelowe, z Kwasem Hialuro...",Mężczyzna,Pielęgnacja,Pielęgnacja twarzy,,,https://www.rossmann.pl/Produkt/Pielegnacja-tw...,4.59,...,NaN,False,"Ingredients: Aqua, Glycerin, Sorbitol, Trehalo...",4.8,19.0,2,szt,1,szt,2.30


In [72]:
# Przegląd podkategorii Mężczyzna > Golenie
df.query('cat_1 == "Mężczyzna" and cat_2 == "Golenie"')['cat_3'].value_counts()

cat_3
Produkty po goleniu                30
Pianki, żele i kremy do golenia     6
Maszynki i trymery do golenia       5
Name: count, dtype: Int64

In [73]:
# Usunięcie kategorii maszynki i trymery do golenia
df = df.query('not(cat_1 == "Mężczyzna" and cat_2 == "Golenie" and cat_3 == "Maszynki i trymery do golenia")')
df.query('cat_1 == "Mężczyzna" and cat_2 == "Golenie"')['cat_3'].value_counts()

cat_3
Produkty po goleniu                30
Pianki, żele i kremy do golenia     6
Name: count, dtype: Int64

In [74]:
# Kategorie pierwszego poziomu
df['cat_1'].value_counts()

cat_1
Mężczyzna                777
Pielęgnacja i higiena    367
Perfumy                  351
Włosy                     82
Lifestyle                  6
Name: count, dtype: Int64

In [75]:
# Na tym etapie mamy wyczyszczone kategorie Mężczyzna i Lifestyle
# Do wyczyszczenia pozostały kategorie "Pielęgnacja i higiena", "Perfumy" i "Włosy"
# Zaczynam od weryfikacji kategorii Perfumy
df[df['cat_1'] == 'Perfumy']['cat_3'].value_counts()

cat_3
Wody toaletowe             269
Wody perfumowane            50
Dezodoranty perfumowane     19
Zestawy zapachowe           13
Name: count, dtype: Int64

In [76]:
df[df['cat_3'] == 'Zestawy zapachowe']

,product_id,name,caption,cat_1,cat_2,cat_3,cat_4,cat_5,url,current_price_pln,...,lowest_price_last_30_days_pln,mega_badge,ingredients,rating,reviews,capacity_value,capacity_unit,price_reference_value,price_reference_unit,price_reference_pln
2,2083791,JAGUAR Era Reloaded,"Zestaw dla mężczyzn, woda toaletowa 100ml + że...",Mężczyzna,Perfumy męskie,Zestawy zapachowe,,,https://www.rossmann.pl/Produkt/Zestawy-zapach...,154.99,...,NaN,False,"Alcohol Denat., Parfum (Fragrance), Aqua (Wate...",4.5,2.0,1,szt,1,szt,154.99
14,2086191,POLICE To Be Born To Shine,"Zestaw dla mężczyzn, woda toaletowa 40ml + żel...",Mężczyzna,Perfumy męskie,Zestawy zapachowe,,,https://www.rossmann.pl/Produkt/Zestawy-zapach...,83.99,...,119.99,False,"Woda toaletowa: Alcohol Denat., Aqua (Water), ...",NaN,NaN,1,szt,1,szt,83.99
70,2087062,HUGO BOSS The Scent,"Zestaw dla mężczyzn, woda toaletowa 100 ml + d...",Mężczyzna,Perfumy męskie,Zestawy zapachowe,,,https://www.rossmann.pl/Produkt/Zestawy-zapach...,429.99,...,NaN,True,"Ingredients: Woda toaletowa: Alcohol Denat., A...",NaN,NaN,1,szt,1,szt,429.99
85,2084303,JEAN PAUL GAULTIER Le Male,"Zestaw dla mężczyzn, woda toaletowa 125ml + ba...",Mężczyzna,Perfumy męskie,Zestawy zapachowe,,,https://www.rossmann.pl/Produkt/Zestawy-zapach...,599.99,...,NaN,True,"Ingredients: Alcohol, Anise Alcohol, Aqua(Wate...",NaN,NaN,1,szt,1,szt,599.99
117,2088899,CRISTIANO RONALDO CR7 Fearless,"Zestaw dla mężczyzn, woda toaletowa 30ml + żel...",Mężczyzna,Perfumy męskie,Zestawy zapachowe,,,https://www.rossmann.pl/Produkt/Zestawy-zapach...,119.99,...,NaN,True,"Alcohol Denat ., Aqua, Parfum, Linalool, Couma...",NaN,NaN,1,szt,1,szt,119.99
127,2105109,CAROLINA HERRERA 212 Men NYC,"Zestaw dla mężczyzn, woda toaletowa 100 ml + ż...",Mężczyzna,Perfumy męskie,Zestawy zapachowe,,,https://www.rossmann.pl/Produkt/Zestawy-zapach...,499.99,...,NaN,True,"Woda toaletowa: Alcohol Denat., Aqua (Water), ...",NaN,NaN,1,szt,1,szt,499.99
150,2074527,HUGO BOSS Hugo Man,"Zestaw dla mężczyzn, woda toaletowa 75 ml + de...",Perfumy,Perfumy męskie,Zestawy zapachowe,,,https://www.rossmann.pl/Produkt/Zestawy-zapach...,249.99,...,NaN,True,"Woda toaletowa 75 ml: Alcohol Denat., Aqua (Wa...",NaN,NaN,1,szt,1,szt,249.99
172,2099189,BVLGARI Man Wood Essence,"Zestaw dla mężczyzn, woda perfumowana 100 ml +...",Mężczyzna,Perfumy męskie,Zestawy zapachowe,,,https://www.rossmann.pl/Produkt/Zestawy-zapach...,649.99,...,NaN,True,Ingredients: Ingredients: Alcohol Denat. (Sd A...,NaN,NaN,1,szt,1,szt,649.99
184,2092169,DOLCE & GABBANA K by Dolce & Gabbana,"Zestaw dla mężczyzn, woda toaletowa 100ml + de...",Mężczyzna,Perfumy męskie,Zestawy zapachowe,,,https://www.rossmann.pl/Produkt/Zestawy-zapach...,399.99,...,NaN,True,"Alcohol, Parfum (Fragrance), Dipropylene Glyco...",NaN,NaN,1,szt,1,szt,399.99
186,2074529,HUGO BOSS The Scent,"Zestaw dla mężczyzn, woda toaletowa 50 ml + de...",Perfumy,Perfumy męskie,Zestawy zapachowe,,,https://www.rossmann.pl/Produkt/Zestawy-zapach...,259.99,...,299.99,True,"Woda toaletowa 50 ml: Alcohol Denat., Aqua/Wat...",NaN,NaN,1,szt,1,szt,259.99


In [77]:
# Na tym etapie zostawiam Zestawy zapachowe. Następną kategorią do weryfikacji jest
# kategoria Włosy
df[df['cat_1'] == 'Włosy']['cat_2'].value_counts()

cat_2
Stylizacja włosów               34
Mycie włosów                    26
Skóra głowy                     16
Koloryzacja włosów               4
Akcesoria i ozdoby do włosów     1
Pielęgnacja włosów               1
Name: count, dtype: Int64

In [78]:
# Usuwam produkty z kategorii Akcesoria i ozdoby do włosów
df = df.query('not(cat_1 == "Włosy" and cat_2 == "Akcesoria i ozdoby do włosów")')
df[df['cat_1'] == 'Włosy']['cat_2'].value_counts()

cat_2
Stylizacja włosów     34
Mycie włosów          26
Skóra głowy           16
Koloryzacja włosów     4
Pielęgnacja włosów     1
Name: count, dtype: Int64

In [79]:
# Weryfikacja kategorii Pielęgnacja i higiena
df[df['cat_1'] == 'Pielęgnacja i higiena']['cat_2'].value_counts()

cat_2
Ciało             203
Broda i zarost    124
Dłonie i stopy     23
Twarz              10
Strefa intymna      6
Senior              1
Name: count, dtype: Int64

In [80]:
# Produkty z powyższej kategorii
df.query('cat_1 == "Pielęgnacja i higiena" and capacity_unit == "szt"')['cat_3'].value_counts()

cat_3
Golenie zarostu                 73
Golenie i depilacja ciała       15
Nietrzymanie moczu               7
Akcesoria do brody i zarostu     5
Zestawy kosmetyków               5
Name: count, dtype: Int64

In [81]:
# Usunięcie produktów powyższych kategorii poza kategorią "Zestawy kosmetyków"
df = df.query('not(cat_1 == "Pielęgnacja i higiena" and capacity_unit == "szt" and cat_3 != "Zestawy kosmetyków")')
df.query('cat_1 == "Pielęgnacja i higiena" and capacity_unit == "szt"')['cat_3'].value_counts()

cat_3
Zestawy kosmetyków    5
Name: count, dtype: Int64

In [82]:
# Przejrzenie kategorii wszystkich pozostałych produktów na sztuki
df.query('capacity_unit == "szt"')['cat_3'].value_counts()

cat_3
Zestawy zapachowe                 58
Koloryzacja włosów i zarostu       6
Zestawy kosmetyków                 5
Serum i wcierki do skóry głowy     4
Pielęgnacja twarzy                 3
Farby do włosów                    3
Oczyszczanie twarzy                1
Pielęgnacja skóry głowy            1
Name: count, dtype: Int64

In [83]:
# Aktualne statystyki df
df.shape

(1482, 21)

In [84]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1482 entries, 0 to 1793
Data columns (total 21 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   product_id                     1482 non-null   int64  
 1   name                           1482 non-null   string 
 2   caption                        1482 non-null   string 
 3   cat_1                          1482 non-null   string 
 4   cat_2                          1482 non-null   string 
 5   cat_3                          1482 non-null   string 
 6   cat_4                          1482 non-null   string 
 7   cat_5                          1482 non-null   string 
 8   url                            1482 non-null   string 
 9   current_price_pln              1482 non-null   float64
 10  regular_price_pln              146 non-null    float64
 11  lowest_price_last_30_days_pln  433 non-null    float64
 12  mega_badge                     1482 non-null   bool  

In [85]:
# Zapisanie przeprocesowanych danych do pliku.
df.to_pickle('../data/male_data_preprocessed.pkl')